In [ ]:
%pip install -U diffusers transformers accelerate peft torch

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

In [ ]:
logo_pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True
).to("cuda")

logo_pipe.load_lora_weights("artificialguybr/LogoRedmond-LogoLoraForSDXL-V2", adapter_name="logo")
logo_pipe.enable_attention_slicing()

In [ ]:
prompt = "A circular logo of a strong man playing basketball, close up, detailed, vibrant and fun colors, LogoRedAF"
batch_size = 10
negative_prompt = ""
scheduler_name = "EulerAncestralDiscreteScheduler"
num_inference_steps = 20
guidance_scale = 5.0
size = 512

In [ ]:
from diffusers import (
    EulerDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
)

def choose_scheduler(scheduler_name):
    match scheduler_name:
        case "EulerDiscreteScheduler":
            return EulerDiscreteScheduler.from_config(logo_pipe.scheduler.config)
        case "EulerAncestralDiscreteScheduler":
            return EulerAncestralDiscreteScheduler.from_config(
                logo_pipe.scheduler.config
            )
        case _:
            raise Exception("Scheduler not supported")()

def run(
    prompt,
    batch_size,
    negative_prompt,
    scheduler_name,
    num_inference_steps,
    guidance_scale,
    width,
    height,
):
    generators = [torch.Generator("cuda").manual_seed(i) for i in range(batch_size)]
    prompts = [prompt] * batch_size
    logo_pipe.scheduler = choose_scheduler(scheduler_name)

    images = logo_pipe(
        prompt=prompts,
        generator=generators,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        width=width,
        height=height,
    ).images

    return images

In [ ]:
from diffusers.utils import make_image_grid

images = run(prompt, batch_size, negative_prompt, scheduler_name, num_inference_steps, guidance_scale, size, size)
make_image_grid(images, 2, 5)